In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1)
import sys
import sklearn
import sklearn.ensemble
%load_ext autoreload
%autoreload 2
from anchor import utils
import lime
import lime.lime_tabular
from int_met import *

C:\Users\youss\AppData\Local\conda\conda\envs\deeplearning\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
dataset_folder = 'Datasets/'
dataset = utils.load_dataset('mortality',discretize=False, balance=True, dataset_folder=dataset_folder)

In [3]:
explainer = lime.lime_tabular.LimeTabularExplainer(dataset.train, feature_names=dataset.feature_names, class_names=dataset.class_target,discretize_continuous=False,)

In [4]:
c = sklearn.ensemble.RandomForestClassifier(n_estimators=70, n_jobs=5, random_state=1)
c.fit(dataset.train, dataset.labels_train)
predict_fn = lambda x: c.predict(x)
print('Train', sklearn.metrics.accuracy_score(dataset.labels_train, predict_fn(dataset.train)))
print('Test', sklearn.metrics.accuracy_score(dataset.labels_test, predict_fn(dataset.test)))

Train 1.0
Test 0.8504784688995215


In [5]:
import pickle
pickling_on = open("rfmort.pickle","wb")
pickle.dump(c, pickling_on)
pickling_on.close()

In [ ]:
dataset.feature_names

In [15]:
exp_fn = lambda i: explainer.explain_instance(dataset.test[i], c.predict_proba, num_features=5, top_labels=1)
def exp_fn_blk(xtest, exp_fn):
    exp1 = []
    for i in range(len(xtest)):
        exp = exp_fn(i)
        exp1.append(exp.as_map()[exp.available_labels()[0]])
    return np.array(exp1)
exp_fn_wrap = lambda x: np.array(exp_fn_blk(x, exp_fn))

In [ ]:
exp1 = exp_fn_blk(dataset.test, exp_fn)

In [7]:
exp1 = exp_fn_blk(dataset.test[:400], exp_fn)
exp2 = exp_fn_blk(dataset.test[:400], exp_fn)

In [9]:
i = calc_identity(exp1,exp2)
print(i)

(100.0, 0, 400)


In [16]:
s = calc_separability(dataset.test[:400], exp_fn_wrap)
print(s)

(0, 390, 152100, 0.0)


In [14]:
a = np.array([1,2,3,4])

In [18]:
a = a[:,np.newaxis]

In [19]:
a.shape

(4, 1)

In [20]:
np.unique(a,axis=0)

array([[1],
       [2],
       [3],
       [4]])

In [11]:
def enc_exp(exp, feature_num):
    enc_exp = np.zeros((len(exp),feature_num))
    for i in range(len(exp)):
        for j in range(len(exp[i])):
            enc_exp[i][int(exp[i,j,0])] = exp[i,j,1]
    return enc_exp

In [12]:
enc1 = enc_exp(exp1, len(dataset.feature_names))
enc2 = enc_exp(exp2, len(dataset.feature_names))

In [13]:
sb = calc_stability(enc1, dataset.labels_test[:400])
print(sb)

C:\Users\youss\AppData\Local\conda\conda\envs\deeplearning\lib\site-packages\sklearn\cluster\k_means_.py:896: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)


(68, 400)
